<font color='purple'><h1>Text features extraction</h1><font>

In [1]:
#ucitavanje osnovnih biblioteka
import pandas as pd
import numpy as np

In [2]:
#ucitavanje Keras biblioteka 
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [3]:
#transformisani podaci
X=pd.read_csv('df_train.csv')
X

,image_id,question,question_id,question_type,multiple_choice_answer,answer_type,image_path
0,458752,What color is the players shirt?,458752002,what color is the,orange,other,train2014/train2014/COCO_train2014_00000045875...
1,458752,Is this man a professional baseball player?,458752003,is this,yes,yes/no,train2014/train2014/COCO_train2014_00000045875...
2,262146,What color is the snow?,262146000,what color is the,white,other,train2014/train2014/COCO_train2014_00000026214...
3,262146,What is the person doing?,262146001,what is the person,skiing,other,train2014/train2014/COCO_train2014_00000026214...
4,262146,What color is the persons headwear?,262146002,what color is the,red,other,train2014/train2014/COCO_train2014_00000026214...
...,...,...,...,...,...,...,...
241772,131067,Is this made of metal?,131067002,is this,yes,yes/no,train2014/train2014/COCO_train2014_00000013106...
241773,524286,What color is the keyboard?,524286001,what color is the,black,other,train2014/train2014/COCO_train2014_00000052428...
241774,524286,Is there a computer mouse on the desk?,524286002,is there a,no,yes/no,train2014/train2014/COCO_train2014_00000052428...
241775,524286,What color is the computer?,524286003,what color is the,black,other,train2014/train2014/COCO_train2014_00000052428...


In [4]:
#prikaz broja jedinstvenih pitanja
questions=X['question']
len(questions.unique())

95095

<font color='blueviolet'><h2>LSTM</h2><font>

Proces:<br>
<li>Tokenizacija rijeci</li>
<li>Padding</li>
<li>Definisanje modela</li>
<li>Izvlacenje features</li>


<h4>Tokenizacija</h4>

In [15]:
#predstavljanje svake rijeci iz pitanja putem tokena
tokenizer=Tokenizer()
tokenizer.fit_on_texts(questions)
sequences=tokenizer.texts_to_sequences(questions)
sequences

[[3, 7, 2, 1, 341, 37],
 [2, 4, 12, 5, 270, 356, 202],
 [3, 7, 2, 1, 151],
 [3, 2, 1, 19, 33],
 [3, 7, 2, 1, 1067, 5876],
 [3, 2, 8, 1, 252, 98],
 [2, 1, 34, 350],
 [2, 1, 34, 85, 40, 5, 189, 100, 52, 208],
 [2, 1, 59, 183],
 [2, 10, 151, 9, 1, 386],
 [2, 1, 145, 141],
 [2, 71, 1056],
 [2, 1, 12, 288],
 [13, 46, 188, 1584, 408, 64, 46, 486],
 [13, 1, 12, 50, 121],
 [2, 4, 23, 8, 7],
 [2, 1, 12, 15, 5, 1345, 188],
 [2725, 106, 1, 362, 74, 4, 61, 8, 5, 5877, 556, 609],
 [3, 201, 6, 130, 8, 4, 18],
 [6, 1, 125, 9, 1, 467],
 [74, 4, 23, 61, 360],
 [13, 1, 177, 20, 5, 1234],
 [3, 2, 4, 12, 114, 9],
 [3, 7, 2, 46, 108],
 [2, 1, 55, 1386, 3558, 46, 309, 393, 1468],
 [3, 7, 2, 1, 206],
 [2, 1, 12, 114, 9, 5, 822, 1290],
 [3, 2, 1, 12, 114, 9],
 [3, 2, 9, 1, 681],
 [6, 1, 275, 955, 8, 5, 6729, 7],
 [2, 1, 535, 2297],
 [6, 1, 380, 1, 65, 483],
 [3, 2, 1, 34, 114, 9],
 [2, 1, 34, 15, 5, 438],
 [2, 16, 5, 229, 80],
 [2, 10, 5, 571, 11, 38, 122],
 [3, 2, 8, 1, 38],
 [2, 1, 101, 9],
 [2, 1, 25, 659]

<h4>Padding<h4>

In [16]:
#produzavanje svake od sekvenci na duzinu najduze
max_length=max(len(sequence) for sequence in sequences) #22
padded_sequences=pad_sequences(sequences,maxlen=max_length)

In [18]:
padded_sequences

array([[  0,   0,   0, ...,   1, 341,  37],
       [  0,   0,   0, ..., 270, 356, 202],
       [  0,   0,   0, ...,   2,   1, 151],
       ...,
       [  0,   0,   0, ...,   9,   1, 312],
       [  0,   0,   0, ...,   2,   1, 154],
       [  0,   0,   0, ...,  36,   5, 187]])

<font color='royalblue'><h3>LSTM1- Embedding nad rjecnikom od rijeci iz nasih pitanja</h3><font>

In [23]:
#rjecnik koji je sastavljen od pitanja iz dataseta
tokenizer.word_index

{'the': 1,
 'is': 2,
 'what': 3,
 'this': 4,
 'a': 5,
 'are': 6,
 'color': 7,
 'in': 8,
 'on': 9,
 'there': 10,
 'of': 11,
 'man': 12,
 'does': 13,
 'to': 14,
 'wearing': 15,
 'it': 16,
 'these': 17,
 'picture': 18,
 'person': 19,
 'have': 20,
 'people': 21,
 'animal': 22,
 'photo': 23,
 'any': 24,
 'room': 25,
 'do': 26,
 'they': 27,
 'you': 28,
 'woman': 29,
 'be': 30,
 'an': 31,
 'he': 32,
 'doing': 33,
 'dog': 34,
 'for': 35,
 'that': 36,
 'shirt': 37,
 'water': 38,
 'cat': 39,
 'at': 40,
 'kind': 41,
 'playing': 42,
 'sport': 43,
 'train': 44,
 'can': 45,
 'his': 46,
 'food': 47,
 'holding': 48,
 'which': 49,
 'look': 50,
 'animals': 51,
 'or': 52,
 'all': 53,
 'made': 54,
 "man's": 55,
 'bus': 56,
 'see': 57,
 'type': 58,
 'sky': 59,
 'where': 60,
 'taken': 61,
 'plate': 62,
 'has': 63,
 'with': 64,
 'same': 65,
 'right': 66,
 'plane': 67,
 'table': 68,
 'sign': 69,
 'eating': 70,
 'she': 71,
 'like': 72,
 'boy': 73,
 'was': 74,
 'sitting': 75,
 'ground': 76,
 'and': 77,
 'being'

In [25]:
#definisanje arhitekture modela
model_lstm1=Sequential()
model_lstm1.add(Embedding(len(tokenizer.word_index)+1,100,input_length=max_length))
model_lstm1.add(LSTM(512,dropout=0.2,return_sequences=True))
model_lstm1.add(LSTM(512,dropout=0.2))


In [35]:
model_lstm1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 22, 100)           1145300   
                                                                 
 lstm (LSTM)                 (None, 22, 512)           1255424   
                                                                 
 lstm_1 (LSTM)               (None, 512)               2099200   
                                                                 
Total params: 4,499,924
Trainable params: 4,499,924
Non-trainable params: 0
_________________________________________________________________


In [26]:
#izvlacenje karakteristika iz pitanja
questions_fetures=model_lstm1.predict(padded_sequences)

7556/7556 [==============================] - 1323s 175ms/step


In [36]:
df_lstm1=pd.DataFrame(questions_fetures)

In [39]:
#cuvanje rezultata u fajl
df_lstm1.to_csv('lstm1_train_all.csv',index=False)

<font color='royalblue'><h3>LSTM2- Embedding pomocu gloVe algoritma</h3></font>

In [42]:
#ucitavanje pretreniranog gloVe algoritma
import gensim.downloader as api
glove_model = api.load('glove-wiki-gigaword-100')

In [43]:
#izgradnja matrice sa tezina iz pretreniranog modela 
vocab_size = len(glove_model.key_to_index)
embedding_dim = glove_model.vector_size
print('Velicina rjecnika:',vocab_size)
print('Dimenzija:',embedding_dim)

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for i, word in enumerate(glove_model.key_to_index):
    embedding_matrix[i] = glove_model.get_vector(word)


Velicina rjecnika: 400000
Dimenzija: 100


In [47]:
#definisanje drugog modela sa definisanim tezinama za encoding
model_lstm2=Sequential()
model_lstm2.add(Embedding(input_dim=vocab_size,output_dim=embedding_dim,weights=[embedding_matrix],input_length=max_length,trainable=False))
model_lstm2.add(LSTM(512,dropout=0.2,return_sequences=True))
model_lstm2.add(LSTM(512,dropout=0.2))

In [48]:
model_lstm2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 22, 100)           40000000  
                                                                 
 lstm_2 (LSTM)               (None, 22, 512)           1255424   
                                                                 
 lstm_3 (LSTM)               (None, 512)               2099200   
                                                                 
Total params: 43,354,624
Trainable params: 3,354,624
Non-trainable params: 40,000,000
_________________________________________________________________


In [49]:
#izvucene karakteristike
questions_features2=model_lstm2.predict(padded_sequences)

7556/7556 [==============================] - 1631s 216ms/step


In [50]:
df_lstm2=pd.DataFrame(questions_features2)

In [52]:
df_lstm2.to_csv('lstm2_train_all.csv',index=False)

<font color='blueviolet'><h2>BERT</h2><font>

Proces:<br>
<li>Tokenizacija</li>
<li>Pretrenirani model</li>
<li>Ekstrakcija</li>

In [5]:
import torch
from transformers import BertTokenizer, BertModel

<h4>Tokenizacija</h4>

In [6]:
#tokenizacija na osnovu pretreniranog modela
#bert-base-uncased: treniran nad tekstovima knjiga,wikippedia,.. sve rijeci su malih slova
#duzina tokenizovane rijeci tj izlaz je 768
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
#pretvaranje svih slova u mala zbog tipa pretreniranog modela
questions_lower=[]
for question in questions:
    questions_lower.append(question.lower())


In [8]:
questions_lower

['what color is the players shirt?',
 'is this man a professional baseball player?',
 'what color is the snow?',
 'what is the person doing?',
 'what color is the persons headwear?',
 "what is in the person's hand?",
 'is the dog waiting?',
 'is the dog looking at a tennis ball or frisbee?',
 'is the sky blue?',
 'is there snow on the mountains?',
 'is the window open?',
 'is she brushing?',
 'is the man smiling?',
 'does his tie pair well with his suit?',
 'does the man look happy?',
 'is this photo in color?',
 'is the man wearing a plain tie?',
 'judging from the dress, was this taken in a latin american country?',
 'what colors are shown in this picture?',
 'are the men on the sidewalk?',
 'was this photo taken recently?',
 'does the guy have a tattoo?',
 'what is this man riding on?',
 'what color is his hat?',
 "is the man's visor providing his face enough protection?",
 'what color is the jacket?',
 'is the man riding on a dirt path?',
 'what is the man riding on?',
 'what is on

In [9]:
inputs = tokenizer_bert(questions_lower, padding=True, truncation=True, return_tensors='pt')

In [10]:
#attention koji govori koliko je koja rijec bitna u zadatom kontekstu
attention_mask=inputs['attention_mask']

In [11]:
attention_mask=attention_mask[4500:7000]

In [12]:
attention_mask.shape

torch.Size([2500, 28])

In [13]:
input_ids = inputs['input_ids']
input_ids=input_ids[4500:7000]

In [14]:
input_ids.shape

torch.Size([2500, 28])

In [15]:
#izvlacenje karakteristika
outputs = bert_model(input_ids,attention_mask)

In [16]:
#izvlacenje posljednjeg skrivenog sloja
last_hidden_state = outputs.last_hidden_state

#Izvlacenje reprezentacije prvog tokena
question_features = last_hidden_state[:, 0, :]

question_features.shape

torch.Size([2500, 768])

In [17]:
question_features_df = pd.DataFrame(question_features.detach().numpy())

In [18]:
#cuvanje rezultata u fajl
question_features_df.to_csv('bert_4500-7000.csv',index=False)

In [22]:
df_bert1=pd.read_csv('bert_1_1000.csv')
df_bert2=pd.read_csv('bert_2_1000.csv')
df_bert3=pd.read_csv('bert_3_1500.csv')

In [25]:
df_bert=pd.concat([df_bert1,df_bert2,df_bert3])

In [27]:
df_bert.to_csv('bert_3500.csv')